Now we're starting to build the iteropt as a system

1. We first define a set of iterations that our iteropt will run through to stack up the memory

In [7]:
import pandas as pd

with open("rd-results-fed-pppsss-include-etf.pkl", "rb") as f:
    result = pd.compat.pickle_compat.load(f)
    
df = result[['first_name', 'last_name', 'ticker', 'return(mean)']]
filtered_df = df[df['return(mean)'] > 0]
df = filtered_df.sort_values(by='return(mean)', ascending=False).reset_index(drop=True)

# Define a function to clean the legislator's name
def clean_name(first_name, last_name):
    # Take the first part of the first name (before any spaces)
    first_name_cleaned = first_name.split()[0].replace(".", "").replace(",", "")
    # Take the first part of the last name (before any spaces or punctuation like '.')
    last_name_cleaned = last_name.split()[0].strip(".").strip(",").strip("'")
    # Combine the cleaned first and last names
    return f"{first_name_cleaned} {last_name_cleaned}"

# Apply the clean_name function to each row in the DataFrame
df['legislator'] = df.apply(lambda row: clean_name(row['first_name'], row['last_name']), axis=1)

df.head()

# number_positive_returns = df[df['return(mean)'] > 0].shape[0]
# print(number_positive_returns)

,first_name,last_name,ticker,return(mean),legislator
0,Ron L,Wyden,AMAT,166.301137,Ron Wyden
1,Ron L,Wyden,KLAC,115.501918,Ron Wyden
2,Angus S,"King, Jr.",QQQ,82.715889,Angus King
3,Ron L,Wyden,AVGO,70.642989,Ron Wyden
4,Sheldon,Whitehouse,QQQ,68.574647,Sheldon Whitehouse


For each iter, it includes:

1. Prediction
    - before pred, we must delete the edges
    - after pred, restore the edges
2. Reflect
    - Put into memory

In [8]:
# shuffle
shuffled_df = df.sample(frac=1).reset_index(drop=True)
shuffled_df.head()

,first_name,last_name,ticker,return(mean),legislator
0,David A,"Perdue , Jr",MSM,1.174379,David Perdue
1,Shelley M,Capito,NVS,2.089110,Shelley Capito
2,Lisa A,Murkowski,QUAL,9.411004,Lisa Murkowski
3,Jerry,"Moran,",V,6.548718,Jerry Moran
4,David A,"Perdue , Jr",IP,16.373285,David Perdue


In [9]:
# instantiate the agent
from agent import agent
from pred import make_pred
from reflect import reflect

# iter over the rows of the df
for index, row in shuffled_df.iterrows():
    ticker = row['ticker']
    legislator = row['legislator']
    print("==================================================")
    print(f"Ticker: {ticker}")
    print(f"Legislator: {legislator}")
    print("Starting prediction...")
    # predict
    pred, log, eval, task_wisdom = make_pred(agent, legislator=legislator, ticker=ticker)
    print(f"Prediction: {pred}")
    print(f"Eval: {eval}")
    # reflect
    print("Starting reflection...")
    reflect(agent, eval, log, task_wisdom, legislator=legislator, ticker=ticker)

    break

Ticker: MSM
Legislator: David Perdue
Starting prediction...
[<Record l.name='David Perdue' r.start_date='2019-09-30' r.end_date='2019-09-30' s.name='MSC Industrial Direct Company Inc.' s.ticker='MSM'>, <Record l.name='David Perdue' r.start_date='2019-07-10' r.end_date='2019-07-10' s.name='MSC Industrial Direct Company Inc.' s.ticker='MSM'>]
[]


> Entering new AgentExecutor chain...
To answer this question, I need to find indirect relationships between David Perdue and MSM. I can start by checking if David Perdue has been assigned to any committees that have been lobbied on by MSM. I can also check if MSM belongs to any NAICS codes that overlap with other companies that David Perdue has engaged in transactions with. I should use the Memory tool to see if there are any similar scenarios from the past that can guide my approach. 

Action: Memory
Action Input: The task involves predicting the likelihood of a financial transaction between a legislator and a company based on indirect relati

/Users/syyun/anaconda3/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/Users/syyun/anaconda3/lib/python3.9/site-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(



Observation: [{'condition': 'When there are no indirect relationships between a legislator and a company, but the legislator has a high overall trading activity and the company has a high overall lobbying activity.', 'action': "In these instances, consider the possibility of a 'BUY_SELL' link, even in the absence of indirect relationships."}, {'condition': 'When a legislator has had financial transactions with companies in the same industry as the company in question.', 'action': 'Check if the legislator has had any financial transactions with companies in the same industry as the company in question. If he has, it suggests a possible interest in that industry, which could increase the likelihood of a transaction with the company.'}, {'condition': 'When a legislator is assigned to a committee that a company has lobbied on a bill related to.', 'action': 'Check for any overlap between the committees that the legislator is assigned to and the bills that the company has lobbied on. If the